In [1]:
import itertools
import collections

In [6]:
testlines = '''RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE'''.splitlines()

In [5]:
with open('day12input.txt') as fp:
    data = fp.read().splitlines()

## Part 1 ##

In [9]:
from collections import deque

def BFS(graph, start):
    visited = set()  # Keep track of the nodes that we've visited
    queue = deque([start])  # Use a queue to implement the BFS

    while queue:
        node = queue.popleft()  # Dequeue a node from front of queue
        if node not in visited:
            visited.add(node)  # Mark the node as visited
            queue.extend(graph[node])  # Enqueue all neighbours
    return visited

In [10]:
def get_graph(lines):
    nbrs = (+1, 0), (-1, 0), (0, +1), (0, -1)
    nrows, ncols = len(lines), len(lines[0])
    graph = {}
    for row, line in enumerate(lines):
        for col, c in enumerate(line):
            graph[(row, col)] = [] # even nodes w/o connections should be here
            for nbr in nbrs:
                nbr_row, nbr_col = row+nbr[0], col+nbr[1]
                if (0 <= nbr_row < nrows) and (0 <= nbr_col < ncols) and (c == lines[nbr_row][nbr_col]):
                    graph[(row, col)].append((nbr_row, nbr_col))
    return graph, nrows, ncols

In [15]:
def get_clusters(graph, nrows, ncols):
    clusters = []
    seen = set()
    for row in range(nrows):
        for col in range(ncols):
            pos = (row, col)
            if pos in seen:
                continue
            cluster = BFS(graph, pos)
            clusters.append(cluster)
            for node in cluster:
                seen.add(node)
    return clusters

In [26]:
sides_from_attached = {0:4, 1:3, 2:2, 3:1, 4:0}
def score_cluster(cluster, graph):
    area = len(cluster)
    perimeter = 0
    for node in cluster:
        num_attached = len(graph[node])
        perimeter += sides_from_attached[num_attached]
    return area*perimeter

In [24]:
def part1(lines):
    graph, nrows, ncols = get_graph(lines)
    clusters = get_clusters(graph, nrows, ncols)
    return sum(score_cluster(cluster, graph) for cluster in clusters)

In [27]:
assert(1930 == part1(testlines))

In [28]:
part1(data)

1374934

## Part 2 ##

One trick is that the number of "sides" is equal to the number of corners, but there are two different types of corners.
See https://www.reddit.com/r/adventofcode/comments/1hcdnk0/comment/m1nkmol/, and verify with examples.

In [29]:
testlines2a = '''AAAA
BBCD
BBCC
EEEC'''.splitlines()
testlines2b = '''EEEEE
EXXXX
EEEEE
EXXXX
EEEEE'''.splitlines()
testlines2c = '''AAAAAA
AAABBA
AAABBA
ABBAAA
ABBAAA
AAAAAA'''.splitlines()

In [32]:
graph, nrows, ncols = get_graph(testlines2a)
clusters = get_clusters(graph, nrows, ncols)

In [33]:
clusters

[{(0, 0), (0, 1), (0, 2), (0, 3)},
 {(1, 0), (1, 1), (2, 0), (2, 1)},
 {(1, 2), (2, 2), (2, 3), (3, 3)},
 {(1, 3)},
 {(3, 0), (3, 1), (3, 2)}]

In [54]:
def get_corners(pos, cluster):
    #  geometry around position A
    #  a b c
    #  h A d
    #  g f e
    row, col = pos
    a = (row-1, col-1) in cluster
    b = (row-1, col) in cluster
    c = (row-1, col+1) in cluster
    d = (row, col+1) in cluster
    e = (row+1, col+1) in cluster
    f = (row+1, col) in cluster
    g = (row+1, col-1) in cluster
    h = (row, col-1) in cluster
    corners = 0
    # outer corners
    if (h == False) and (b == False):
        corners += 1
    if (b == False) and (d == False):
        corners += 1
    if (d == False) and (f == False):
        corners += 1
    if (f == False) and (h == False):
        corners += 1
    # inner corners
    if (h == True) and (b == True) and (a == False):
        corners += 1
    if (b == True) and (d == True) and (c == False):
        corners += 1
    if (d == True) and (f == True) and (e == False):
        corners += 1
    if (f == True) and (h == True) and (g == False):
        corners += 1
    return corners

In [56]:
def score_cluster2(cluster):
    area = len(cluster)
    sides = sum(get_corners(node, cluster) for node in cluster)
    return area*sides

In [58]:
def part2(lines):
    graph, nrows, ncols = get_graph(lines)
    clusters = get_clusters(graph, nrows, ncols)
    return sum(score_cluster2(cluster) for cluster in clusters)

In [60]:
assert(236 == part2(testlines2b))
assert(368 == part2(testlines2c))

In [61]:
part2(data)

841078

In [46]:
clusters

[{(0, 0), (0, 1), (0, 2), (0, 3)},
 {(1, 0), (1, 1), (2, 0), (2, 1)},
 {(1, 2), (2, 2), (2, 3), (3, 3)},
 {(1, 3)},
 {(3, 0), (3, 1), (3, 2)}]